In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
import numpy as np
from statsmodels.stats import inter_rater as irr
pd.options.display.max_colwidth = 240

In [2]:
juanjo = pd.read_csv('../Data/Data_Annotated/Anotaciones_juanjo.csv')
juanma = pd.read_csv('../Data/Data_Annotated/Anotaciones_juanma.csv')
ger = pd.read_csv('../Data/Data_Annotated/anotaciones_ger.csv')

In [3]:
juanjo = juanjo[['text', 'tweet_id', 'sentiment']]
juanjo = juanjo.rename(columns = {'sentiment':'sentiment_juanjo'})
juanma = juanma[['text', 'tweet_id', 'sentiment']]
juanma = juanma.rename(columns = {'sentiment':'sentiment_juanma'})
ger = ger[['text', 'tweet_id', 'sentiment']]
ger = ger.rename(columns = {'sentiment':'sentiment_ger'})

In [4]:
inner_merged_total = pd.merge(juanjo, juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id'])

In [5]:
inner_merged_total_2 = pd.merge(inner_merged_total, ger[['tweet_id', 'sentiment_ger']], on=['tweet_id'])

In [7]:
inner_merged_total_2 = inner_merged_total_2.fillna('Otra')

In [9]:
le = preprocessing.LabelEncoder()
le.fit(list(set(list(inner_merged_total_2['sentiment_juanjo'].unique()) + 
    list(inner_merged_total_2['sentiment_juanma'].unique())+ 
    list(inner_merged_total_2['sentiment_ger'].unique()))))
juanjo_encoded = le.transform(inner_merged_total_2['sentiment_juanjo'])
juanma_encoded = le.transform(inner_merged_total_2['sentiment_juanma'])
ger_encoded = le.transform(inner_merged_total_2['sentiment_ger'])
arr = np.array((juanjo_encoded, juanma_encoded, ger_encoded)).T
agg = irr.aggregate_raters(arr)
irr.fleiss_kappa(agg[0], method='fleiss')

0.1879288164665523

In [2]:
juanjo = pd.read_csv('../Data/Data_Annotated/new_juanjo_annotations.csv')
juanma = pd.read_csv('../Data/Data_Annotated/new_juanma_annotations.csv')

In [3]:
juanjo = juanjo.rename(columns = {'sentimento':'sentiment_juanjo', 'relevancia': 'relevancia_juanjo', 'emocion': 'emocion_juanjo'})
juanma = juanma.rename(columns = {'sentimento':'sentiment_juanma', 'relevancia': 'relevancia_juanma', 'emocion': 'emocion_juanma'})

In [4]:
merged_df = pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']], juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id'])
merged_relevancia = pd.merge(juanjo[['text', 'tweet_id', 'relevancia_juanjo']], juanma[['tweet_id', 'relevancia_juanma']], on=['tweet_id'])
merged_emocion = pd.merge(juanjo[['text', 'tweet_id', 'emocion_juanjo']], juanma[['tweet_id', 'emocion_juanma']], on=['tweet_id'])

In [22]:
merged_relevancia_no = merged_relevancia[(merged_relevancia['relevancia_juanjo'] == 'No') | (merged_relevancia['relevancia_juanma'] == 'No')]

In [24]:
merged_relevancia_no

,text,tweet_id,relevancia_juanjo,relevancia_juanma
15,@RICHENAO De hecho solo lo harían unas pocas familias a las que ud les lame el culo. #ElDebateDefinitivo #ElCambioEnPrimera.,1529351840772546565,Si,No
16,"Quiero enviar un saludo a todos los ciclistas del país, que sigan haciendo deporte y que por favor sean prudentes a la hora de manejar por las vias de Colombia. Lo digo porque Egan Bernal se dio un golpe muy fuerte en la cabeza y aún no se recupera. #YaEsSuficiente",1529602483055640577,No,No
28,@darcyquinnr #LeApuestoA Dios a la fe a la esperanza a la vida a la coherencia,1529060822269448194,No,Si
63,#ElDebateDefinitivo viendo a Fico sin una palabra significativa. #YoVotoPorFajardo por sus palabras sensatas y posturas claras. El único sin lugares comunes!,1528903235209150465,No,Si
65,Para quienes han migrado de internet fijo de @Tigo_Colombia a @MovistarCo en Medellín en los últimos días/meses... ¿Qué tal les ha ido? #MeInquieta,1528806026643968002,No,No
76,"@emiliolagos1 #MiVotoEsPetroYFrancia Quiero llegar a los 1.000 seguidores. Estamos a puertas de un cambio histórico #ProgresistaSigueProgresista Devolveré el follow a cada uno, vamos a crecer en comunidad, así le hacemos frente a la desinformación entre todos. ME AYUDAN!! 🔄",1528849330932695040,Si,No
90,Patria Honor lealtad !! #FedericoEsColombia,1528503220875714560,Si,No


In [7]:
dict_emociones = {
'Confianza' : ['Confianza', 'Aceptación', 'Admiración','Agrado'],
'miedo' : ['Miedo'],
'sorpresa' : ['Sorpresa', 'Incertidumbre', 'Asombro'],
'tristeza' : ['Tristeza'],
'asco' : ['Asco', 'Desagrado', 'Odio', 'Decepción', 'Indiferencia'],
'ira' : ['Ira'],
'expectativa' : ['Expectativa', 'Vigilancia'],
'alegria' : ['Alegria', 'Calma'],
'otra' : ['Otra'] }

In [8]:
for i  in dict_emociones.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_emociones[i]))

In [9]:
merged_df = merged_df.fillna(False)

In [10]:
merged_df['labels'] = merged_df.apply(lambda x: ', '.join(x.index[x == True]), axis=1)

In [11]:
raw_counts = {}
for i in dict_emociones.keys():
    for j in dict_emociones[i]:
        raw_emotion = j
        raw_counts[j] = len(merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]) 

In [23]:
raw_emotion = 'Agrado'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels
0,"#NosUnimosONosJodemos es una persona de fiar, no tiene rabo de paja, es un buen administrador y gestor, une, da confianza y hace la politica que es a favor del pueblo, es temeroso de Dios.",1528357012517621762,"{""choices"": [""Admiración"", ""Agrado""]}","{""choices"": [""Admiración"", ""Agrado"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
2,"@SaraLop88370165 @FicoGutierrez La propuesta de Fico está basada y planificada, el tiene la capacidad de asumir la presidencia #FicoGanador #ElMejorCandidatoEs",1529826360180346880,Agrado,"{""choices"": [""Admiración"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
5,"@enriqueyimis1 @WRadioColombia #ElMejorCandidatoEs de lejos la dupla Petro y Francia. Porque se convirtieron en el referente del país que nos incluye a todos. No solo los que somos abiertamente antifascistas, sino a los ciudadanos comun...",1529785193804050433,Agrado,"{""choices"": [""Admiración"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
7,"#MiVotoEsPorFICO #SantandereanosConFICO porq conectará el campo triplicando la inversión en vías terciarias para que nuestros campesinos puedan sacar sus productos a los mercados #FicoElGanador Nuevo simulador de votación,eligiendo a @...",1529933656093081636,Agrado,Confianza,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
20,#VotoPor Sergio Fajardo. Un presidente decente y que no divida es lo que necesita Colombia,1529613037866459136,Agrado,Admiración,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
63,#ElDebateDefinitivo viendo a Fico sin una palabra significativa. #YoVotoPorFajardo por sus palabras sensatas y posturas claras. El único sin lugares comunes!,1528903235209150465,Agrado,"{""choices"": [""Admiración"", ""Sorpresa""]}",True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo, sorpresa_juanma"
80,"Isssss @petrogustavo llamará a la ONU para que investigue la corrupción, se va a caer toda la estantería de la corrupción en Colombia. #MiVotoEsPetroYFrancia",1528543164067459072,"{""choices"": [""Expectativa"", ""Agrado""]}","{""choices"": [""Expectativa"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,True,True,False,False,False,False,"Confianza_juanma, Confianza_juanjo, expectativa_juanma, expectativa_juanjo"
90,Patria Honor lealtad !! #FedericoEsColombia,1528503220875714560,Agrado,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Confianza_juanjo


In [12]:
raw_counts

{'Confianza': 34,
 'Aceptación': 1,
 'Admiración': 16,
 'Agrado': 8,
 'Miedo': 8,
 'Sorpresa': 3,
 'Incertidumbre': 6,
 'Asombro': 2,
 'Tristeza': 1,
 'Asco': 22,
 'Desagrado': 30,
 'Odio': 6,
 'Decepción': 17,
 'Indiferencia': 0,
 'Ira': 8,
 'Expectativa': 15,
 'Vigilancia': 0,
 'Alegria': 5,
 'Calma': 1,
 'Otra': 6}

In [130]:
raw_emotion = 'Aceptación'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels
62,"Fajardo comienza el debate mostrando como las agresiones y señalamientos entre Fico y Petro generan una afectación a la población colombiana, afectación en la cual él no quiere participar #YoVotoPorFajardo #ElDebateDefinitivo #DebateEl...",1528905779373391872,Aceptación,Admiración,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"


In [131]:
algo = 'otra'
merged_df[(merged_df[algo + '_juanma'] == True) & (merged_df[algo + '_juanjo'] == True)]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels


In [13]:
df_list = []
emotions_count = {}
for i in dict_emociones.keys():
    emotion_df = merged_df[(merged_df[i + '_juanma']== True) | 
                           (merged_df[i + '_juanjo']== True)][['text', 'labels', 'tweet_id' , i + '_juanma', i +'_juanjo', 'sentiment_juanma', 'sentiment_juanjo']]
    emotion_df = emotion_df.rename(columns= {i + '_juanma': 'juanma', i +'_juanjo': 'juanjo'})
    emotion_df['emocion'] = i
    emotions_count[i] = len(emotion_df)
    df_list.append(emotion_df)
full_df = pd.concat(df_list, ignore_index=True)
full_df_disagreement = full_df[full_df['juanma'] != full_df['juanjo']]
full_df_agreement = full_df[full_df['juanma'] == full_df['juanjo']]

In [12]:
len(full_df_disagreement)

82

In [18]:
len(full_df_agreement.drop_duplicates(subset=['tweet_id']))

58

In [14]:
full_df_disagreement[full_df_disagreement['emocion'] == 'sorpresa']

,text,labels,tweet_id,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion
49,#AColombiaLePreocupa la propuesta de pensiones de Petro. Ayer en #ElDebateDefinitivo quedó nuevamente confirmado que si piensa usar la ahorros de millones para gastárselo en sus promesas de campaña.,"miedo_juanjo, sorpresa_juanma, ira_juanjo",1529119564503691264,True,False,Incertidumbre,"{""choices"": [""Ira"", ""Miedo""]}",sorpresa
50,#AColombiaLePreocupa que después de una pandemia salimos llenos de rencor,"sorpresa_juanma, asco_juanjo",1529096515356176385,True,False,Incertidumbre,Decepción,sorpresa
51,"#AColombiaLePreocupa el abandono de las regiones! Fortalecer la seguridad alimentaria, salud, educación, vivienda, vías decentes el fortalecimiento del agro en su conjunto!","sorpresa_juanjo, expectativa_juanma, expectativa_juanjo",1529089553923026946,False,True,Expectativa,"{""choices"": [""Expectativa"", ""Incertidumbre""]}",sorpresa
52,La Silicon Valley Latinoamericana aún no tiene software para las votaciones. #AColombiaLePreocupa,"sorpresa_juanma, asco_juanjo",1529070396087185409,True,False,Incertidumbre,Decepción,sorpresa
53,"Fajardo tiene la misma propuesta de petro en pensiones, y ni el mismo lo sabe. Petro se lo tuvo que explicar. #ElDebateDefinitivo","sorpresa_juanma, asco_juanma, asco_juanjo",1528936500745973760,True,False,"{""choices"": [""Decepción"", ""Sorpresa""]}",Desagrado,sorpresa
55,"#ElDebateDefinitivo le falta a Federico Gutiérrez decir que ""vamos a vivir sabroso"" y ya se termina de robar todo el discurso de Petro. Consejo. Si alguien se le roba la idea al otro. No vayan por el que se roba la idea, vayan por el q...","sorpresa_juanjo, asco_juanma, asco_juanjo",1528921475553669128,False,True,"{""choices"": [""Decepción"", ""Desagrado""]}","{""choices"": [""Asombro"", ""Asco""]}",sorpresa
56,#ElDebateDefinitivo viendo a Fico sin una palabra significativa. #YoVotoPorFajardo por sus palabras sensatas y posturas claras. El único sin lugares comunes!,"Confianza_juanma, Confianza_juanjo, sorpresa_juanma",1528903235209150465,True,False,"{""choices"": [""Admiración"", ""Sorpresa""]}",Agrado,sorpresa
57,#MeInquieta que @AndresPastrana_ esté generando miedo e inquietud y... posible violencia el día de elecciones si no queda el candidato que cada uno quiere. Y @FiscaliaCol @PoliciaColombia @PGN_COL no dicen nada. Estos últimos son corre...,"miedo_juanjo, sorpresa_juanma, asco_juanma, expectativa_juanma",1528704766028697601,True,False,"{""choices"": [""Expectativa"", ""Desagrado"", ""Incertidumbre""]}",Miedo,sorpresa
58,"AVANZA EL PLAN GOLPE A LA DEMOCRACIA, EL NARCOESTADO PARAMILITAR Y SU MAFIA CORRUPTA NO DESEAN PERDER EL PODER #MiVotoEsPetroYFrancia https://t.co/riWZi2IP4G","miedo_juanma, miedo_juanjo, sorpresa_juanma, asco_juanma, ira_juanjo",1528510805007900672,True,False,"{""choices"": [""Desagrado"", ""Miedo"", ""Incertidumbre""]}","{""choices"": [""Ira"", ""Miedo""]}",sorpresa
59,"@FicoGutierrez #FicoPropone @FicoGutierrez #FedericoEsColombia así es,nada nos detiene,ni el agua,ni el granizo impidió que Medellín se volcara para para acompañár y escuchár al escogído por los paisas y millones de Colombianos ¡Domingo...","Confianza_juanma, sorpresa_juanjo, alegria_juanjo",1528513296575434752,False,True,Confianza,"{""choices"": [""Asombro"", ""Alegria""]}",sorpresa


In [138]:
full_df_agreement[full_df_agreement['emocion'] == 'alegria']

,text,labels,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion


In [ ]:
full_df[full_df['emocion'].str.contains('')]

In [10]:
merged_df.to_csv('../Data/Data_Annotated/merged_annotations.csv', index=False)

In [140]:
merged_relevancia = merged_relevancia.fillna('Si')
merged_relevancia['relevancia_juanjo_bool'] = merged_relevancia['relevancia_juanjo'].map({'Si': True, 'No': False})
merged_relevancia['relevancia_juanma_bool'] = merged_relevancia['relevancia_juanma'].map({'Si': True, 'No': False})

In [75]:
arr = np.array((merged_relevancia['relevancia_juanjo_bool'], merged_relevancia['relevancia_juanma_bool'])).T
agg = irr.aggregate_raters(arr)
irr.fleiss_kappa(agg[0], method='fleiss')

0.4179894179894169

In [79]:
merged_emocion = merged_emocion.fillna('No')
merged_emocion['emocion_juanjo_bool'] = merged_emocion['emocion_juanjo'].map({'Si': True, 'No': False})
merged_emocion['emocion_juanma_bool'] = merged_emocion['emocion_juanma'].map({'Si': True, 'No': False})

In [94]:
arr_num = np.array((merged_emocion['emocion_juanjo_numerica'], merged_emocion['emocion_juanma_numerica'])).T
agg_num = irr.aggregate_raters(arr_num)
irr.fleiss_kappa(agg_num[0], method='fleiss')

0.16904482121873435

In [88]:
agreements = {}
for i in dict_emociones.keys():
    arr = np.array((merged_df[i + '_juanjo'], merged_df[i + '_juanma'])).T
    agg = irr.aggregate_raters(arr)
    agreements[i] = irr.fleiss_kappa(agg[0], method='fleiss')

In [89]:
agreements

{'Confianza': 0.6616155861911815,
 'miedo': 0.18518518518518337,
 'sorpresa': 0.11290322580645148,
 'tristeza': -0.005076142131970106,
 'asco': 0.6156169111302849,
 'ira': -0.042105263157894146,
 'expectativa': 0.3596001176124661,
 'alegria': -0.03125000000000165,
 'otra': -0.03125000000000165}

In [92]:
total = sum(emotions_count.values())
avgd_agreement = 0
mean_agreement = 0
for i in agreements.keys():
    avgd_agreement+= agreements[i]*(emotions_count[i]/total)
    mean_agreement+= agreements[i]
mean_agreement = mean_agreement/len(agreements)

In [68]:
avgd_agreement

0.44378135901472815

In [93]:
mean_agreement

0.20280440229285557

In [19]:
pd.options.display.max_colwidth = 300
merged_emocion_disagreement = merged_emocion[(merged_emocion['emocion_juanjo'] == 'No') |(merged_emocion['emocion_juanma'] == 'No')]

In [30]:
to_exclude = merged_relevancia_no[['text','tweet_id']].append(merged_emocion_disagreement[['text','tweet_id']], ignore_index=True).drop_duplicates(subset=['tweet_id'])

In [56]:
disagreement_unique = full_df_disagreement.drop_duplicates(subset=['tweet_id'])[['text','tweet_id']]
disagreement_unique = disagreement_unique.reset_index(drop = True)

In [61]:
df_clean = disagreement_unique[disagreement_unique.merge(to_exclude, how='left', on='tweet_id')['text_y'].isna()]

In [77]:
df_final = df_clean[~df_clean['text'].str.contains('https')]
df_final.to_csv('../Data/Data_Annotated/to_annotate.csv', index=False)

In [79]:
df_final

,text,tweet_id
0,#FicoElGanador y con el gana Colombia.,1529850058417618946
1,Ya convencí a 10 personas #ConvenceUnoMásPorPetro,1529573334907768832
4,Es claro quien es el presidente y quienes van a quedar rabones ! Petro los peinó tranquilo ! #ElDebateDefinitivo,1529144849441644545
7,#ElDebateDefinitivo ¿Ya están diciendo que el cafre Petro los peinó a todos?,1528903851981553666
8,Ya no quieroas de los mismo #MiVotoEsPetroYFrancia,1528557117858357249
10,"Los únicos que deben de tener miedo a un cambio de Gustavo Petro son los corruptos, narcotraFICO, paramilitares y los vividores del estado que perderán sus privilegios!! #MiVotoEsPetroYFrancia",1528497422695030786
15,"#YaEsSuficiente ya es suficiente de @petrogustavo , espero que perdiendo estas elecciones nos deje en paz a los colombianos con su proyecto fracasado comunista. Ya mk ya por favor 🙏🏻",1529466881609957378
16,#AColombiaLePreocupa la propuesta de pensiones de Petro. Ayer en #ElDebateDefinitivo quedó nuevamente confirmado que si piensa usar la ahorros de millones para gastárselo en sus promesas de campaña.,1529119564503691264
18,"#ElDebateDefinitivo mi voto es por @FicoGutierrez. Jamás votaré por el que intento asesinar a mi padre, un guerrillero que sólo quiere el poder para perpetuarse. Ciegos los que piensan que @petrogustavo es el mesías.",1528898629708169217
19,#MeInquieta que @AndresPastrana_ esté generando miedo e inquietud y... posible violencia el día de elecciones si no queda el candidato que cada uno quiere. Y @FiscaliaCol @PoliciaColombia @PGN_COL no dicen nada. Estos últimos son corresponsables de lo que pueda pasar el donimgo,1528704766028697601
